In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/memory-test-on-drugged-islanders-data/Islander_data.csv


In [35]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings(action='ignore')

In [36]:
df = pd.read_csv("/kaggle/input/memory-test-on-drugged-islanders-data/Islander_data.csv")

In [37]:
df.head()

,first_name,last_name,age,Happy_Sad_group,Dosage,Drug,Mem_Score_Before,Mem_Score_After,Diff
0,Bastian,Carrasco,25,H,1,A,63.5,61.2,-2.3
1,Evan,Carrasco,52,S,1,A,41.6,40.7,-0.9
2,Florencia,Carrasco,29,H,1,A,59.7,55.1,-4.6
3,Holly,Carrasco,50,S,1,A,51.7,51.2,-0.5
4,Justin,Carrasco,52,H,1,A,47.0,47.1,0.1


In [38]:
df.shape

(198, 9)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   first_name        198 non-null    object 
 1   last_name         198 non-null    object 
 2   age               198 non-null    int64  
 3   Happy_Sad_group   198 non-null    object 
 4   Dosage            198 non-null    int64  
 5   Drug              198 non-null    object 
 6   Mem_Score_Before  198 non-null    float64
 7   Mem_Score_After   198 non-null    float64
 8   Diff              198 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 14.1+ KB


In [40]:
df.describe()

,age,Dosage,Mem_Score_Before,Mem_Score_After,Diff
count,198.000000,198.000000,198.000000,198.000000,198.000000
mean,39.530303,1.989899,57.967677,60.922222,2.954545
std,12.023099,0.818504,15.766007,18.133851,10.754603
min,24.000000,1.000000,27.200000,27.100000,-40.400000
25%,30.000000,1.000000,46.525000,47.175000,-3.175000
50%,37.000000,2.000000,54.800000,56.750000,1.700000
75%,48.000000,3.000000,68.400000,73.250000,5.925000
max,83.000000,3.000000,110.000000,120.000000,49.000000


# Data Preprocessing

In [41]:
{column : len(X[column].unique()) for column in X.columns}

{'first_name': 139,
 'last_name': 18,
 'age': 45,
 'Happy_Sad_group': 2,
 'Dosage': 3,
 'Drug': 3,
 'Mem_Score_Before': 162,
 'Mem_Score_After': 151,
 'Diff': 142}

In [42]:
def onehot_encode(df, column):
    df = df.copy()
    
    dummies = pd.get_dummies(df[column], prefix=column)
    
    if len(df[column].unique()) == 2:
        dummies = dummies.drop(dummies.columns[0], axis=1)
        
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    
    return df

In [43]:
def preprocessing_inputs(df) :
    df = df.copy()

    # One hot encode Categorical features 

    for column in ['first_name' , 'last_name' , 'Happy_Sad_group'] :
        df = onehot_encode(df , column=column)

    # split data 

    X = df.drop('Drug'  , axis =1 )
    y = df['Drug']

    # Train_test_split 
    X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state= 42)

    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test


In [44]:
X_train, X_test, y_train, y_test = preprocessing_inputs(df)


In [45]:
X_train

,age,Dosage,Mem_Score_Before,Mem_Score_After,Diff,first_name_Aaron,first_name_Adam,first_name_Ai,first_name_Akane,first_name_Akira,...,last_name_Lopez,last_name_McCarthy,last_name_Morin,last_name_Novak,last_name_Price,last_name_Rodriguez,last_name_Steiner,last_name_Summers,last_name_Takahashi,Happy_Sad_group_S
137,1.809496,-1.228994,2.353206,2.271692,0.325790,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,-1.0
5,-0.212430,-1.228994,0.500713,-0.161282,-1.048796,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,1.0
66,-0.212430,1.260507,-0.319140,0.786414,1.854176,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,2.788867,1.0
138,1.162480,-1.228994,0.244118,0.320879,0.181603,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,1.0
104,-0.859447,0.015756,-1.245386,-1.059099,0.075866,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,5.531727,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1.566865,0.015756,-0.131387,-0.244413,-0.222122,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,3.847077,-0.259938,-0.358569,1.0
14,-1.102078,-1.228994,-1.007566,-1.563428,-1.164146,0.0,-0.079809,12.529964,-0.113228,-0.079809,...,2.788867,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,1.0
92,-1.021201,0.015756,-0.269072,-0.083692,0.268115,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,1.0
179,-0.940324,1.260507,-0.544443,-1.092351,-1.058408,0.0,-0.079809,-0.079809,-0.113228,-0.079809,...,-0.358569,-0.180775,-0.079809,-0.113228,-0.079809,-0.113228,-0.259938,-0.259938,-0.358569,1.0


In [46]:
y_train.value_counts()


Drug
A    55
T    52
S    51
Name: count, dtype: int64

# Model Training 

In [47]:
models = {
    "                   Logistic Regression": LogisticRegression(),
    "                   K-Nearest Neighbors": KNeighborsClassifier(),
    "                         Decision Tree": DecisionTreeClassifier(),
    "Support Vector Machine (Linear Kernel)": LinearSVC(),
    "   Support Vector Machine (RBF Kernel)": SVC(),
    "                        Neural Network": MLPClassifier(),
    "                         Random Forest": RandomForestClassifier(),
    "                     Gradient Boosting": GradientBoostingClassifier(),
    "                               XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "                              LightGBM": LGBMClassifier(),
    "                              CatBoost": CatBoostClassifier(verbose=0)
}


# Encode target labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


# Now train using y_train_encoded
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                   Logistic Regression trained.
                   K-Nearest Neighbors trained.
                         Decision Tree trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                        Neural Network trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000050 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 195
[LightGBM] [Info] Number of data points in the train set: 158, number of used features: 8
[LightGBM] [Info] Start training from score -1.055262
[LightGBM] [Info] Start training from score -1.130769
[LightGBM] [Info] Start training from score -1.111351
[LightGBM] [Warning] No further splits with positive gain

# Results

In [48]:
for name, model in models.items():
    print(name + ": {:.2f}%".format(model.score(X_test, y_test) * 100))

                   Logistic Regression: 40.00%
                   K-Nearest Neighbors: 35.00%
                         Decision Tree: 40.00%
Support Vector Machine (Linear Kernel): 35.00%
   Support Vector Machine (RBF Kernel): 45.00%
                        Neural Network: 37.50%
                         Random Forest: 45.00%
                     Gradient Boosting: 55.00%
                               XGBoost: 40.00%
                              LightGBM: 40.00%
                              CatBoost: 47.50%


In [49]:
from sklearn.metrics import confusion_matrix, classification_report

# Track best model
best_model_name = None
best_model = None
best_score = 0

# Find best model
for name, model in models.items():
    score = model.score(X_test, y_test)
    print(f"{name}: {score * 100:.2f}%")
    if score > best_score:
        best_score = score
        best_model_name = name
        best_model = model

# Print best model
print(f"\n Best Model: {best_model_name} with accuracy: {best_score * 100:.2f}%")

# Predictions
y_pred = best_model.predict(X_test)

# Confusion Matrix
print("\n Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report
print("\n Classification Report:")
print(classification_report(y_test, y_pred))

                   Logistic Regression: 40.00%
                   K-Nearest Neighbors: 35.00%
                         Decision Tree: 40.00%
Support Vector Machine (Linear Kernel): 35.00%
   Support Vector Machine (RBF Kernel): 45.00%
                        Neural Network: 37.50%
                         Random Forest: 45.00%
                     Gradient Boosting: 55.00%
                               XGBoost: 40.00%
                              LightGBM: 40.00%
                              CatBoost: 47.50%

 Best Model:                      Gradient Boosting with accuracy: 55.00%

 Confusion Matrix:
[[7 2 3]
 [3 7 5]
 [2 3 8]]

 Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.58      0.58        12
           1       0.58      0.47      0.52        15
           2       0.50      0.62      0.55        13

    accuracy                           0.55        40
   macro avg       0.56      0.56      0.55        40
weighted 

# Hyper parameter Tuning 

In [50]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5],
}

gb_model = GradientBoostingClassifier(random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=gb_model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1)

# Run GridSearchCV
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'min_samples_split': [2, 5],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [51]:
print(" Best Parameters:", grid_search.best_params_)

 Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 300, 'subsample': 0.8}


In [52]:
# Use the best estimator
best_gb_model = grid_search.best_estimator_
y_pred = best_gb_model.predict(X_test)

y_pred

array([0, 2, 1, 2, 1, 1, 2, 1, 0, 2, 1, 1, 0, 0, 2, 0, 2, 0, 2, 1, 2, 0,
       0, 0, 1, 0, 2, 0, 2, 0, 0, 1, 1, 1, 2, 1, 2, 2, 2, 0])

In [58]:
from sklearn.metrics import accuracy_score

print("\n Accuracy :" , accuracy_score(y_test , y_pred))

print("\n Confusion Matrix after tuning:")
print(confusion_matrix(y_test, y_pred))

print("\n Classification Report after tuning:")
print(classification_report(y_test, y_pred))


 Accuracy : 0.45

 Confusion Matrix after tuning:
[[7 3 2]
 [2 6 7]
 [5 3 5]]

 Classification Report after tuning:
              precision    recall  f1-score   support

           0       0.50      0.58      0.54        12
           1       0.50      0.40      0.44        15
           2       0.36      0.38      0.37        13

    accuracy                           0.45        40
   macro avg       0.45      0.46      0.45        40
weighted avg       0.45      0.45      0.45        40

